<a href="https://colab.research.google.com/github/ajuliasousa/TCC-2025-1/blob/main/TCC_Modelos_Tradicionais_DL_Dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Panorama Geral do Trabalho**

**Título:**

Análise Comparativa de Modelos Clássicos e
Deep Learning na Detecção de Discurso de Ódio

**Objetivo Geral:**

Avaliar e comparar o desempenho de diferentes algoritmos de classificação, incluindo modelos tradicionais de Machine Learning (ML) e arquiteturas de Deep Learning (DL), na tarefa de detecção de discurso de ódio em dados textuais, analisando o impacto de distintas abordagens de representação vetorial e técnicas de balanceamento de classes.

**Objetivos Específicos:**

1.	Realizar o pré-processamento e a limpeza de um corpus textual rotulado para discurso de ódio.
2.	Representar os dados textuais por meio de TF-IDF e embeddings (pré-treinados ou ajustados).
3.	Treinar e avaliar modelos de ML com TF-IDF: Naive Bayes, Regressão Logística, SVM, Random Forest, LightGBM, MLP.
4.	Preparar dados com tokenização, vocabulário e padding para modelos DL com embeddings.
5.	Treinar redes profundas: CNN e LSTM com embeddings.
6.	Aplicar diferentes estratégias de balanceamento de dados: oversampling, undersampling e uso de class_weight.
7.	Avaliar os modelos usando métricas: F1-score ponderado, AUC e acurácia.
8.	Comparar sistematicamente o desempenho dos modelos em diferentes cenários e técnicas de representação.
9.	Discutir os impactos do tipo de vetorização e do balanceamento nos resultados obtidos.


### **Dataset 1: Hate Speech and Offensive Language Dataset**

**Link:** https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset

Este dataset reúne textos coletados do Twitter com o objetivo de auxiliar pesquisas em detecção automática de discurso de ódio e linguagem ofensiva. Ele foi desenvolvido por pesquisadores da Universidade de Cornell e da Universidade da Califórnia e publicado originalmente no artigo "Automated Hate Speech Detection and the Problem of Offensive Language" (Davidson et al., 2017).

Cada tweet da base foi anotado manualmente e classificado em uma das seguintes categorias:

    Hate Speech (Discurso de ódio): conteúdo que expressa ódio ou incita violência contra um grupo com base em atributos como raça, etnia, nacionalidade, gênero, orientação sexual, religião, entre outros.

    Offensive Language (Linguagem ofensiva): conteúdo que pode conter palavrões, xingamentos ou linguagem agressiva, mas que não necessariamente se configura como discurso de ódio.

    Neither (Nenhum dos dois): textos que não se enquadram em nenhuma das categorias acima, sendo neutros ou não ofensivos.

O dataset é composto por 24.783 tweets, sendo uma ferramenta valiosa para tarefas de processamento de linguagem natural (PLN), como classificação de texto, análise de sentimentos e desenvolvimento de sistemas de moderação automatizada.

### **Bibliotecas e Visão do Dataset**

In [ ]:
# bibliotecas
import pandas as pd
import numpy as np

# drive
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/TCC/Datasets/Hate Speech and Offensive Language Dataset/labeled_data.csv'
df = pd.read_csv(file_path)

Mounted at /content/drive


In [ ]:
# informações básicas
print("Primeiras linhas do dataset:")
print(df.head())
print("\nInformações do dataset:")
df.info()
print("\nDescrição estatística das colunas numéricas:")
print(df.describe())
print("\nDistribuição da coluna 'class':")
print(df['class'].value_counts(normalize=True) * 100)

Primeiras linhas do dataset:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------          

A distribuição das classes mostra que a maior parte dos tweets foi rotulada como linguagem ofensiva (77,4%), seguida por neutros (16,8%), e apenas uma pequena parte corresponde a discurso de ódio (5,8%).

Além da coluna de texto (tweet), o dataset traz colunas com contagens indicando o número de votos recebidos para cada categoria durante o processo de anotação manual. Isso permite avaliar o nível de concordância entre os anotadores.
Estrutura dos dados:

    7 colunas no total, sendo 6 numéricas e 1 de texto.

    Todas as linhas estão completas (sem valores ausentes).

    O tamanho total da base é de aproximadamente 1,3 MB.

Algumas estatísticas:

    A maioria dos tweets recebeu 3 votos durante a anotação.

    A média de votos para "linguagem ofensiva" é significativamente maior do que para "discurso de ódio" e "neutro".

    A classificação final (class) é derivada da categoria com mais votos em cada tweet.

Esse panorama inicial ajuda a entender o viés de distribuição e reforça a importância de cuidados ao treinar modelos com essa base, já que o número reduzido de exemplos de discurso de ódio pode influenciar o desempenho do classificador.

### **Pré-processamento Textual**

**1. Limpeza com clean_text()**

Função que aplica regras para "limpar" os textos brutos:

    Remove URLs: tira links da internet (ex: http://...).

    Remove menções: elimina nomes de usuários (@usuario).

    Remove hashtags: exclui palavras precedidas de # (ou poderia apenas remover o símbolo).

    Remove caracteres não alfabéticos: exclui números, pontuações e símbolos, mantendo letras e acentuação.

    Converte para minúsculas: uniformiza o texto.

    Remove espaços extras: com strip().

Resultado: uma versão mais "limpa" do tweet.

**2. Tokenização e Processamento Avançado**

Com a função preprocess_text_advanced():

    Tokenização: divide o texto em palavras (tokens) com word_tokenize.

    Remoção de stopwords: elimina palavras muito comuns em inglês (ex: the, and, is) que pouco contribuem para a análise.

    Lematização: reduz as palavras à sua forma base (ex: running vira run, cars vira car), usando o WordNetLemmatizer.

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# regras de limpeza
def clean_text(text):
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text) # Remove menções
    text = re.sub(r'#\w+', '', text) # Remove hashtags (ou pode mantê-las sem o #)
    text = re.sub(r'[^a-zA-Z\sÀ-ÿ]', '', text, re.I|re.A) # Remove caracteres não alfabéticos, mantém acentos
    text = text.lower() # Converte para minúsculas
    text = text.strip()
    return text

# Aplicar limpeza
df['cleaned_tweet'] = df['tweet'].apply(clean_text)

# Baixar recursos necessários para stop words e lematização
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Inicializar lematizador e stop words para inglês
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english')) # <--- Aqui a mudança para inglês

def preprocess_text_advanced(text):
    tokens = word_tokenize(text)
    # Remover stop words e lematizar
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return processed_tokens

# Aplicar pré-processamento avançado
df['processed_tokens'] = df['cleaned_tweet'].apply(preprocess_text_advanced)

print("\nExemplo de tokens processados:")
print(df[['tweet', 'processed_tokens']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Exemplo de tokens processados:
                                               tweet  \
0  !!! RT @mayasolovely: As a woman you shouldn't...   
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   

                                    processed_tokens  
0  [rt, woman, shouldnt, complain, cleaning, hous...  
1  [rt, boy, dat, coldtyga, dwn, bad, cuffin, dat...  
2  [rt, dawg, rt, ever, fuck, bitch, start, cry, ...  
3                           [rt, look, like, tranny]  
4  [rt, shit, hear, might, true, might, faker, bi...  


### **Vetorização com  TF-IDF**

O texto processado dos tweets foi transformado em uma matriz numérica utilizando a técnica TF-IDF (Term Frequency–Inverse Document Frequency), que atribui pesos às palavras com base na sua frequência e relevância nos textos.

Para isso, os tokens foram reunidos novamente em strings e, em seguida, vetorizados com o TfidfVectorizer, limitando a 5.000 as palavras mais representativas. O resultado é uma matriz com 24.783 linhas (um para cada tweet) e 5.000 colunas (palavras-chave), que serve como base para treinar modelos de machine learning.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 'processed_tokens' é a coluna com as listas de tokens
# TfidfVectorizer espera strings como entrada, então junta-se os tokens de volta
df['processed_text'] = df['processed_tokens'].apply(lambda tokens: ' '.join(tokens))

# inicializa o TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# aplica o vetorizador aos textos processados
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_text'])

print("\nForma da matriz TF-IDF:")
print(tfidf_matrix.shape)


Forma da matriz TF-IDF:
(24783, 5000)


### **Aplicação em dataset desbalanceado**

#### **Divisão em treino/ teste**

Os dados vetorizados foram divididos em dois conjuntos: 80% para treino e 20% para teste, garantindo reprodutibilidade com random_state=42. Essa divisão permite treinar o modelo em uma parte dos dados e avaliar seu desempenho em dados não vistos.

In [ ]:
from sklearn.model_selection import train_test_split

# define as features (X) e o target (y)
X = tfidf_matrix
y = df['class']

# divide os dados em conjuntos de treino e teste
# test_size=0.20 significa 20% dos dados para teste
# random_state para reprodutibilidade
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("Forma dos dados de treino (X_train, y_train):", X_train.shape, y_train.shape)
print("Forma dos dados de teste (X_test, y_test):", X_test.shape, y_test.shape)

Forma dos dados de treino (X_train, y_train): (19826, 5000) (19826,)
Forma dos dados de teste (X_test, y_test): (4957, 5000) (4957,)


#### **Modelos Tradicionais**

Quatro modelos de classificação supervisionada foram treinados para prever categorias de tweets (discurso de ódio, linguagem ofensiva ou nenhum dos dois), utilizando a matriz TF-IDF como entrada. Os modelos testados foram: **Regressão Logística, Naive Bayes Multinomial, Support Vector Machine (SVM com kernel linear) e Random Forest**. Cada modelo foi ajustado com os dados de treino e avaliado com os dados de teste usando métricas como **acurácia, F1-score (ponderado) e AUC (curva ROC, multiclasse, ponderada)**. Os resultados foram armazenados em um dicionário para facilitar a comparação de desempenho entre os classificadores.

**Regressão Logística**

A Regressão Logística foi utilizada como um modelo linear de base para classificação multiclasse. Ela estima a probabilidade de um tweet pertencer a cada uma das três classes com base nas palavras mais relevantes (extraídas via TF-IDF). O modelo foi treinado com um número maior de iterações (max_iter=1000) para garantir a convergência, dado o tamanho da matriz. Por oferecer suporte ao método predict_proba, foi possível calcular a métrica AUC ponderada (one-vs-rest), o que fornece uma medida robusta da capacidade do modelo em distinguir entre as classes.

**Naive Bayes Multinomial**

O modelo Naive Bayes Multinomial é especialmente adequado para tarefas de classificação de texto, por assumir que as características (neste caso, palavras) ocorrem de forma independente. Ele é simples, eficiente e frequentemente usado como forte baseline em PLN. No experimento, ele também permitiu a geração de probabilidades de classe (predict_proba), o que possibilitou o cálculo da AUC multiclasse. Apesar de suas suposições simplificadas, o Naive Bayes costuma ter desempenho competitivo quando os dados estão bem vetorizados.

**Support Vector Machine (SVM)**

O SVM foi utilizado com kernel linear, uma configuração comum e eficaz para dados textuais de alta dimensionalidade, como é o caso da matriz TF-IDF. Foi ativada a opção probability=True para possibilitar o cálculo da AUC, embora isso torne o treinamento mais custoso computacionalmente. O SVM busca encontrar hiperplanos que melhor separam as classes, sendo especialmente útil quando há margens claras entre categorias. Apesar de não ser naturalmente probabilístico, sua robustez o torna uma escolha frequente em tarefas de classificação com múltiplas classes.

**Random Forest**

O modelo Random Forest foi treinado com 100 árvores de decisão, combinando os resultados de várias árvores para aumentar a estabilidade e a precisão da predição. Como um modelo de ensemble, ele lida bem com dados complexos e é menos sensível a overfitting do que uma única árvore. Também oferece suporte a predict_proba, permitindo calcular a AUC ponderada. Sua capacidade de capturar interações não lineares entre os termos dos textos pode ser vantajosa em relação a modelos lineares, especialmente quando o texto contém padrões mais sutis.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

# Inicializar um dicionário para armazenar os resultados de todos os modelos
results = {}

# 1. Regressão Logística
print("Treinando Regressão Logística...")
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

# Obter probabilidades para AUC
# O SVM por padrão não tem predict_proba, a menos que probability=True seja definido,
# o que pode ser computacionalmente caro. Para SVM, AUC é menos comum ou requer adaptações.
# Para outros modelos como Logistic Regression, Naive Bayes e Random Forest, predict_proba funciona.

try:
    lr_probabilities = lr_model.predict_proba(X_test)
    # Calcule AUC para cada classe e depois a média (macro, weighted, etc.)
    # Para classificação multiclasse, AUC é tipicamente calculado 'ovr' (one-vs-rest)
    # Usaremos 'ovr' com average='weighted' para considerar o desbalanceamento
    lr_auc = roc_auc_score(y_test, lr_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lr_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação da Regressão Logística:")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))
print("F1-score ponderado:", f1_score(y_test, lr_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", lr_auc)

# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions, average='weighted'),
    'auc (weighted_ovr)': lr_auc
}

print("-" * 50)

# 2. Naive Bayes Multinomial
print("Treinando Naive Bayes Multinomial...")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)

try:
    nb_probabilities = nb_model.predict_proba(X_test)
    nb_auc = roc_auc_score(y_test, nb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    nb_auc = "N/A (predict_proba não disponível ou com problema)"

print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))
print("F1-score ponderado:", f1_score(y_test, nb_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", nb_auc)

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions, average='weighted'),
    'auc (weighted_ovr)': nb_auc
}


print("-" * 50)

# 3. Support Vector Machine (SVM)
# Para AUC com SVM, adicionar probability=True
svm_model = SVC(kernel='linear', probability=True)
print("Treinando SVM (Kernel Linear)...")
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

try:
    # verificação
    if hasattr(svm_model, 'predict_proba'):
        svm_probabilities = svm_model.predict_proba(X_test)
        svm_auc = roc_auc_score(y_test, svm_probabilities, multi_class='ovr', average='weighted')
    else:
         svm_auc = "N/A (probability=False)"
except AttributeError:
    svm_auc = "N/A (problema ao obter predict_proba)"


print("Avaliação do SVM:")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))
print("F1-score ponderado:", f1_score(y_test, svm_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", svm_auc)

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions, average='weighted'),
    'auc (weighted_ovr)': svm_auc
}


print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # seta o número de árvores
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

try:
    rf_probabilities = rf_model.predict_proba(X_test)
    rf_auc = roc_auc_score(y_test, rf_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    rf_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação do Random Forest:")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))
print("F1-score ponderado:", f1_score(y_test, rf_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", rf_auc)

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions, average='weighted'),
    'auc (weighted_ovr)': rf_auc
}

Treinando Regressão Logística...
Avaliação da Regressão Logística:
              precision    recall  f1-score   support

           0       0.53      0.17      0.25       290
           1       0.91      0.96      0.94      3832
           2       0.83      0.83      0.83       835

    accuracy                           0.89      4957
   macro avg       0.76      0.65      0.67      4957
weighted avg       0.88      0.89      0.88      4957

Acurácia:  0.8922735525519467
F1-score ponderado: 0.8784331676581796
AUC (ponderado, one-vs-rest): 0.9386849954267578
--------------------------------------------------
Treinando Naive Bayes Multinomial...
Avaliação do Naive Bayes Multinomial:
              precision    recall  f1-score   support

           0       0.50      0.00      0.01       290
           1       0.83      0.99      0.90      3832
           2       0.90      0.38      0.53       835

    accuracy                           0.83      4957
   macro avg       0.74      0.46   

Além dos modelos tradicionais, também foram implementados dois modelos avançados para comparação de desempenho: **LightGBM e uma Rede Neural densa (MLP).**

O **LightGBM**, um algoritmo de **gradient boosting** eficiente e otimizado para velocidade, foi treinado diretamente sobre a matriz TF-IDF esparsa, com configuração para classificação multiclasse. Por suportar `predict_proba`, permitiu o cálculo da métrica AUC multiclasse ponderada, além de métricas tradicionais como acurácia e F1-score.

Já a **Rede Neural MLP**, construída com o **Keras/TensorFlow**, possui camadas densas com funções de ativação ReLU, camadas de dropout para reduzir overfitting e uma saída com softmax para previsão de múltiplas classes. O modelo foi treinado por 20 épocas, com validação interna durante o processo, e sua performance foi avaliada também com base em métricas clássicas, incluindo a AUC gerada a partir das probabilidades preditas.

Ambas as abordagens complementam os modelos anteriores, oferecendo perspectivas mais robustas sobre o comportamento dos dados e o desempenho de classificadores baseados em vetores TF-IDF.

In [ ]:
# instalações
!pip install tensorflow

# bibliotecas para CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
# --- Modelos Avançados com TF-IDF ---

# 5. LightGBM (Gradient Boosting)

import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score # Importar métricas

print("\n" + "="*50)
print("Treinando LightGBM...")

# converte matriz esparsa para o formato LightGBM
# LightGBM funciona diretamente com matrizes esparsas do SciPy
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=len(df['class'].unique()), random_state=42)

lgb_model.fit(X_train, y_train) # uso do X_train e y_train do split anterior (baseado em TF-IDF)

lgb_predictions = lgb_model.predict(X_test)

# obtém probabilidades para AUC
try:
    lgb_probabilities = lgb_model.predict_proba(X_test)
    lgb_auc = roc_auc_score(y_test, lgb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lgb_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação do LightGBM:")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))
print("F1-score ponderado:", f1_score(y_test, lgb_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", lgb_auc)

# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions, average='weighted'),
    'auc (weighted_ovr)': lgb_auc
}


print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras

# reutiliza TensorFlow importado anteriormente para a CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

print("Treinando Rede Neural Densa (MLP)...")

# A matriz TF-IDF já está pronta (X_train, X_test, y_train, y_test)

# constrói o Modelo MLP
mlp_model = Sequential([
    # flatten é necessário se a entrada não for 1D, mas a matriz TF-IDF já é 2D (samples x features)
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)), # camada densa com neurônios
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # camada de saída (igual à CNN para classificação multiclasse)
    Dense(len(df['class'].unique()), activation='softmax')
])


mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy', # loss para caso de labels inteiras
                  metrics=['accuracy'])

mlp_model.summary()

# treina o Modelo MLP
epochs_mlp = 20
batch_size_mlp = 64

print("\nTreinando MLP...")
history_mlp = mlp_model.fit(X_train, y_train,
                          epochs=epochs_mlp,
                          batch_size=batch_size_mlp,
                          validation_split=0.1, # parte dos dados de treino para validação
                          verbose=1)

# avalia o Modelo MLP
print("\nAvaliando MLP no conjunto de teste...")
loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP no conjunto de teste: {accuracy_mlp:.4f}")

# classification report
mlp_predictions_probs = mlp_model.predict(X_test)
mlp_predictions = tf.argmax(mlp_predictions_probs, axis=1).numpy()

# obtém probabilidades para AUC (TensorFlow/Keras model.predict retorna probabilidades com softmax)
try:
    # model.predict() em Keras com ativação 'softmax' na última camada já retorna probabilidades
    mlp_auc = roc_auc_score(y_test, mlp_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e: # tratamento de exceção
    mlp_auc = f"N/A (problema ao calcular AUC: {e})"


print("\nAvaliação completa da MLP:")
print(classification_report(y_test, mlp_predictions))
print("F1-score ponderado:", f1_score(y_test, mlp_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", mlp_auc)


# dicionário de resultados
mlp_report = classification_report(y_test, mlp_predictions, output_dict=True)
results['MLP'] = {
    'accuracy': accuracy_score(y_test, mlp_predictions),
    'precision (macro)': mlp_report['macro avg']['precision'],
    'recall (macro)': mlp_report['macro avg']['recall'],
    'f1-score (macro)': mlp_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, mlp_predictions, average='weighted'),
    'auc (weighted_ovr)': mlp_auc
}


print("="*50)


Treinando LightGBM...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.164951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 30288
[LightGBM] [Info] Number of data points in the train set: 19826, number of used features: 979
[LightGBM] [Info] Start training from score -2.855966
[LightGBM] [Info] Start training from score -0.255358
[LightGBM] [Info] Start training from score -1.784623


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM:
              precision    recall  f1-score   support

           0       0.45      0.20      0.28       290
           1       0.93      0.94      0.94      3832
           2       0.80      0.92      0.86       835

    accuracy                           0.90      4957
   macro avg       0.73      0.69      0.69      4957
weighted avg       0.88      0.90      0.89      4957

Acurácia:  0.8963082509582408
F1-score ponderado: 0.886614172266831
AUC (ponderado, one-vs-rest): 0.9453254006688412
--------------------------------------------------
Treinando Rede Neural Densa (MLP)...


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 256)            │     1,280,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,539 (5.01 MB)

 Trainable params: 1,313,539 (5.01 MB)

 Non-trainable params: 0 (0.00 B)


Treinando MLP...
Epoch 1/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.7889 - loss: 0.6339 - val_accuracy: 0.8926 - val_loss: 0.2952
Epoch 2/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.9041 - loss: 0.2659 - val_accuracy: 0.8986 - val_loss: 0.2846
Epoch 3/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.9315 - loss: 0.1901 - val_accuracy: 0.8921 - val_loss: 0.3081
Epoch 4/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.9460 - loss: 0.1489 - val_accuracy: 0.8981 - val_loss: 0.3556
Epoch 5/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.9592 - loss: 0.1216 - val_accuracy: 0.8886 - val_loss: 0.3699
Epoch 6/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - accuracy: 0.9712 - loss: 0.0923 - val_accuracy: 0.8775 - val_loss: 0.4229
Epoch 7/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.9772 - loss: 0.0733 - val_accuracy: 0.8845 - val_loss: 0.4753
Epoch 8/20
279/279 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.9782 -

#### **Modelos de DL**

Para explorar abordagens mais profundas de aprendizado, foi implementada **uma rede neural convolucional (CNN)** voltada para a classificação de textos. Como esse tipo de modelo trabalha melhor com sequências de palavras em vez de vetores TF-IDF, os tweets foram tokenizados e convertidos em **sequências inteiras**, com padding para garantir um comprimento fixo.

O modelo foi construído com uma camada de embedding (para mapear palavras em vetores densos), seguida por uma **camada convolucional 1D** que captura padrões locais no texto e uma **camada de pooling** que extrai as informações mais relevantes. Camadas densas e dropout foram adicionadas para refinar o aprendizado e reduzir overfitting. O modelo foi treinado por 10 épocas e avaliado com base em métricas como acurácia, F1-score e AUC multiclasse. Essa arquitetura permite capturar melhor a estrutura local e semântica dos textos, sendo especialmente útil para dados curtos e ruidosos como tweets.

In [ ]:
# reutiliza importações de bibliotecas
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

# --- Pré-processamento para CNN ---

# O TfidfVectorizer não é ideal para CNNs, precisamos de uma representação baseada em sequências/embeddings.

# 1. Tokenização para CNN (Sequências de Inteiros)
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_text'])

sequences = tokenizer.texts_to_sequences(df['processed_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded:")
print(padded_sequences[:5])
print("\nForma das sequências padded:", padded_sequences.shape)

# --- prepara labels para CNN ---
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(df['class'])

print("\nLabels originais:", df['class'].unique())
print("Labels codificadas:", encoded_y)

# --- divide dados para CNN (usando as sequências e labels codificadas) ---
X_cnn = padded_sequences
y_cnn = encoded_y

X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.20, random_state=42)

print("\nForma dos dados de treino para CNN:", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de teste para CNN:", X_test_cnn.shape, y_test_cnn.shape)

# --- constrói o Modelo CNN ---
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



# --- treina o Modelo CNN---
epochs = 10
batch_size = 32

print("\nTreinando CNN sem class_weight...")
history = model.fit(X_train_cnn, y_train_cnn,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    verbose=1)

model.summary()

# --- avaliação do modelo
print("\nAvaliando CNN no conjunto de teste...")
loss, accuracy = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\nAcurácia da CNN no conjunto de teste: {accuracy:.4f}")

# classification report para CNN
cnn_predictions_probs = model.predict(X_test_cnn)
cnn_predictions = tf.argmax(cnn_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da CNN:")
print(classification_report(y_test_cnn, cnn_predictions))

# cálculo F1-score ponderado e AUC para a CNN
cnn_f1_weighted = f1_score(y_test_cnn, cnn_predictions, average='weighted')

try:
    cnn_auc = roc_auc_score(y_test_cnn, cnn_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    cnn_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (CNN):", cnn_f1_weighted)
print("AUC (ponderado, one-vs-rest) (CNN):", cnn_auc)

# dicionário de resultados
try:
    cnn_report = classification_report(y_test_cnn, cnn_predictions, output_dict=True)
    results['CNN (Embedding+Seq, No Class_Weight)'] = {
        'accuracy': accuracy_score(y_test_cnn, cnn_predictions),
        'precision (macro)': cnn_report['macro avg']['precision'],
        'recall (macro)': cnn_report['macro avg']['recall'],
        'f1-score (macro)': cnn_report['macro avg']['f1-score'],
        'f1-score (weighted)': cnn_f1_weighted,
        'auc (weighted_ovr)': cnn_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da CNN (sem class_weight) não foram armazenados.")


Exemplo de sequências padded:
[[   3  100  838  920 2932  204   20   38   84   71   17    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3   96   78 8148 5799   37 2045   78    4  457  381    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


558/558 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.8300 - loss: 0.4777 - val_accuracy: 0.9097 - val_loss: 0.2735
Epoch 2/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 26s 47ms/step - accuracy: 0.9209 - loss: 0.2217 - val_accuracy: 0.9037 - val_loss: 0.2652
Epoch 3/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 26s 46ms/step - accuracy: 0.9549 - loss: 0.1342 - val_accuracy: 0.9057 - val_loss: 0.3110
Epoch 4/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 46ms/step - accuracy: 0.9787 - loss: 0.0648 - val_accuracy: 0.9052 - val_loss: 0.3900
Epoch 5/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.9877 - loss: 0.0405 - val_accuracy: 0.8865 - val_loss: 0.4609
Epoch 6/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - accuracy: 0.9888 - loss: 0.0322 - val_accuracy: 0.8916 - val_loss: 0.5410
Epoch 7/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 42s 44ms/step - accuracy: 0.9924 - loss: 0.0237 - val_accuracy: 0.8906 - val_loss: 0.5615
Epoch 8/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9936 - loss: 0.0183 - val_accurac

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 98, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,038,539 (15.41 MB)

 Trainable params: 1,346,179 (5.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,692,360 (10.27 MB)


Avaliando CNN no conjunto de teste...

Acurácia da CNN no conjunto de teste: 0.8745
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step

Avaliação completa da CNN:
              precision    recall  f1-score   support

           0       0.39      0.23      0.29       290
           1       0.91      0.94      0.93      3832
           2       0.79      0.80      0.80       835

    accuracy                           0.87      4957
   macro avg       0.70      0.66      0.67      4957
weighted avg       0.86      0.87      0.87      4957

F1-score ponderado (CNN): 0.8673805816983572
AUC (ponderado, one-vs-rest) (CNN): 0.9169551714873231


O modelo baseado em **LSTM (Long Short-Term Memory)** foi desenvolvido como uma abordagem voltada ao reconhecimento de padrões sequenciais nos tweets. Utilizando a mesma base de dados preparada para a CNN, os textos foram previamente tokenizados e convertidos em sequências numéricas com padding, garantindo comprimento uniforme.

A arquitetura do modelo inclui uma **camada de embedding**, que transforma cada palavra em um vetor denso, seguida por uma camada **LSTM com 128 unidades**, capaz de capturar dependências de longo prazo no texto. Foram adicionadas camadas densas e de **dropout** para auxiliar na generalização do modelo. A saída utiliza a função softmax para classificação multiclasse.

O modelo foi treinado por 15 épocas e avaliado por meio de métricas como acurácia, F1-score ponderado e AUC, demonstrando seu potencial para lidar com a natureza sequencial e contextual das mensagens de texto. Essa abordagem é especialmente adequada para capturar a ordem das palavras, o que pode ser relevante na detecção de nuances ofensivas em linguagem natural.

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
# from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalMaxPooling1D, Conv1D
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score # Já importado

# --- Reutiliza dados preparados para a CNN ---
# X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn
# max_words, max_sequence_length, embedding_dim
# label_encoder (fitado)

# --- Constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

model_lstm = Sequential([
    # camada de embedding: Reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # camada LSTM
    # return_sequences=False significa que a LSTM retorna a saída da última etapa de tempo (para classificação de sequência)
    # return_sequences=True seria para tarefas de predição de sequência ou para empilhar LSTMs
    LSTM(128),

    # camada de dropout para evitar overfitting
    Dropout(0.5),

    # camada densa (Fully Connected)
    Dense(64, activation='relu'), # camada densa adicional
    Dropout(0.5),

    # camada de saída (igual à CNN)
    Dense(len(label_encoder.classes_), activation='softmax')
])


model_lstm.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model_lstm.summary()

# --- treina o Modelo LSTM  ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM sem class_weight...")
history_lstm = model_lstm.fit(X_train_cnn, y_train_cnn, # usa os mesmos dados preparados para CNN
                              epochs=epochs_lstm,
                              batch_size=batch_size_lstm,
                              validation_split=0.1,
                              verbose=1)


# --- avaliação do modelo
print("\nAvaliando LSTM no conjunto de teste...")
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\nAcurácia da LSTM no conjunto de teste: {accuracy_lstm:.4f}")

# classification report para LSTM
lstm_predictions_probs = model_lstm.predict(X_test_cnn)
lstm_predictions = tf.argmax(lstm_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da LSTM:")
# lembrete que y_test_cnn são labels codificadas (0, 1, 2)
class_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test_cnn, lstm_predictions, target_names=class_names)) #

# cálculo de F1-score ponderado e AUC para a LSTM
lstm_f1_weighted = f1_score(y_test_cnn, lstm_predictions, average='weighted')

try:
    lstm_auc = roc_auc_score(y_test_cnn, lstm_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    lstm_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (LSTM):", lstm_f1_weighted)
print("AUC (ponderado, one-vs-rest) (LSTM):", lstm_auc)


print("="*50)

# dicionário de resultados
try:
    lstm_report = classification_report(y_test_cnn, lstm_predictions, output_dict=True, target_names=class_names)
    results['LSTM (Embedding+Seq, No Class_Weight)'] = {
        'accuracy': accuracy_score(y_test_cnn, lstm_predictions),
        'precision (macro)': lstm_report['macro avg']['precision'],
        'recall (macro)': lstm_report['macro avg']['recall'],
        'f1-score (macro)': lstm_report['macro avg']['f1-score'],
        'f1-score (weighted)': lstm_f1_weighted,
        'auc (weighted_ovr)': lstm_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da LSTM (sem class_weight) não foram armazenados.")


Construindo o Modelo LSTM...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando LSTM sem class_weight...
Epoch 1/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 119s 208ms/step - accuracy: 0.7655 - loss: 0.7198 - val_accuracy: 0.7796 - val_loss: 0.6582
Epoch 2/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 170s 258ms/step - accuracy: 0.7748 - loss: 0.6770 - val_accuracy: 0.7796 - val_loss: 0.6490
Epoch 3/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 168s 197ms/step - accuracy: 0.7752 - loss: 0.6738 - val_accuracy: 0.7796 - val_loss: 0.6474
Epoch 4/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 152s 215ms/step - accuracy: 0.7752 - loss: 0.6675 - val_accuracy: 0.7796 - val_loss: 0.6472
Epoch 5/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 131s 196ms/step - accuracy: 0.7682 - loss: 0.6822 - val_accuracy: 0.7796 - val_loss: 0.6480
Epoch 6/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 142s 197ms/step - accuracy: 0.7762 - loss: 0.6638 - val_accuracy: 0.7796 - val_loss: 0.6483
Epoch 7/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 145s 203ms/step - accuracy: 0.7729 - loss: 0.6678 - val_accuracy: 0.7796 - val_loss: 0.6479
Epoch 8/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

### **Balanceamento do dataset e retreino dos modelos**

#### **Class_weight**

Nesta etapa do experimento, foi aplicado o **balanceamento de classes por meio do parâmetro class_weight**, com o objetivo de mitigar o impacto do desbalanceamento dos dados na performance dos modelos.

Utilizando a função `compute_class_weight` da biblioteca `sklearn`, foram calculados pesos proporcionais à frequência das classes no conjunto de treinamento. Esses pesos foram incorporados diretamente ao processo de treinamento em modelos que suportam esse recurso, como **Regressão Logística, SVM, Random Forest, LightGBM e Redes Neurais com Keras**.

A inclusão dos pesos penaliza erros cometidos em classes minoritárias, forçando os algoritmos a considerarem com maior atenção esses exemplos menos frequentes. Essa estratégia é especialmente eficaz em cenários de classificação desbalanceada, como o de detecção de discursos ofensivos, contribuindo para **melhorias em métricas como recall e F1-score das classes minoritárias, sem necessariamente comprometer a acurácia geral**.

Modelos como o Naive Bayes Multinomial, que não oferecem suporte direto a class_weight, foram mantidos como referência sem ajuste neste momento.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# X e y já estão definidos e representam a matriz TF-IDF e os rótulos originais.

print("Forma dos dados de treino (X_train, y_train):", X_train.shape, y_train.shape)
print("Forma dos dados de teste (X_test, y_test):", X_test.shape, y_test.shape)

# --- cálculo de Class Weights ---
# uso do y_train para calcular os pesos, pois o balanceamento deve ser baseado nos dados de treino.
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

print("\nPesos das classes calculados (baseado em y_train):")
print(class_weight_dict)

# 1. Regressão Logística
print("\n" + "="*50)
print("Treinando Regressão Logística com class_weight...")
lr_model = LogisticRegression(max_iter=1000, class_weight=class_weight_dict)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

# probabilidades para AUC
try:
    lr_probabilities = lr_model.predict_proba(X_test)
    lr_auc = roc_auc_score(y_test, lr_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lr_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação da Regressão Logística:")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))
print("F1-score ponderado:", f1_score(y_test, lr_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", lr_auc)

# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression (Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions, average='weighted'),
    'auc (weighted_ovr)': lr_auc
}

print("-" * 50)

# 2. Naive Bayes Multinomial
# Naive Bayes Multinomial no scikit-learn não tem o argumento class_weight diretamente.
# O treino será sem class_weight, como é o padrão para este modelo.

print("Treinando Naive Bayes Multinomial (sem class_weight)...")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)

try:
    nb_probabilities = nb_model.predict_proba(X_test)
    nb_auc = roc_auc_score(y_test, nb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    nb_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))
print("F1-score ponderado:", f1_score(y_test, nb_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", nb_auc)

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB (No Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions, average='weighted'),
    'auc (weighted_ovr)': nb_auc
}

print("-" * 50)

# 3. Support Vector Machine (SVM)

print("Treinando SVM (Kernel Linear) com class_weight...")

svm_model = SVC(kernel='linear', class_weight=class_weight_dict, probability=True)

svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

# cálculo de AUC para SVM com probability=True
try:
    if hasattr(svm_model, 'predict_proba'):
        svm_probabilities = svm_model.predict_proba(X_test)
        svm_auc = roc_auc_score(y_test, svm_probabilities, multi_class='ovr', average='weighted')
    else:
         svm_auc = "N/A (probability=False)"
except AttributeError:
    svm_auc = "N/A (problema ao obter predict_proba)"


print("Avaliação do SVM:")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))
print("F1-score ponderado:", f1_score(y_test, svm_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", svm_auc)

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM (Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions, average='weighted'),
    'auc (weighted_ovr)': svm_auc
}


print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest com class_weight...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

try:
    rf_probabilities = rf_model.predict_proba(X_test)
    rf_auc = roc_auc_score(y_test, rf_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    rf_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação do Random Forest:")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))
print("F1-score ponderado:", f1_score(y_test, rf_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", rf_auc)

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest (Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions, average='weighted'),
    'auc (weighted_ovr)': rf_auc
}

print("-" * 50)

# 5. LightGBM (Gradient Boosting)
print("Treinando LightGBM com class_weight...")
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=len(df['class'].unique()), random_state=42, class_weight=class_weight_dict)

lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)

try:
    lgb_probabilities = lgb_model.predict_proba(X_test)
    lgb_auc = roc_auc_score(y_test, lgb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lgb_auc = "N/A (predict_proba não disponível ou com problema)"


print("Avaliação do LightGBM:")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))
print("F1-score ponderado:", f1_score(y_test, lgb_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", lgb_auc)

# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM (Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions, average='weighted'),
    'auc (weighted_ovr)': lgb_auc
}

print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras
print("Treinando Rede Neural Densa (MLP) com class_weight...")

# constrói do Modelo MLP (estrutura igual)
mlp_model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(df['class'].unique()), activation='softmax')
])

mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

mlp_model.summary()

# treina o Modelo MLP
epochs_mlp = 20
batch_size_mlp = 64

print("\nTreinando MLP com class_weight...")
history_mlp = mlp_model.fit(X_train, y_train,
                          epochs=epochs_mlp,
                          batch_size=batch_size_mlp,
                          validation_split=0.1,
                          verbose=1,
                          class_weight=class_weight_dict)
mlp_model.summary()

# avaliação do modelo
print("\nAvaliando MLP no conjunto de teste original...")
loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP no conjunto de teste original: {accuracy_mlp:.4f}")

# classification report para MLP
mlp_predictions_probs = mlp_model.predict(X_test)
mlp_predictions = tf.argmax(mlp_predictions_probs, axis=1).numpy()

try:
    mlp_auc = roc_auc_score(y_test, mlp_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    mlp_auc = f"N/A (problema ao calcular AUC: {e})"

print("\nAvaliação completa da MLP:")
print(classification_report(y_test, mlp_predictions))
print("F1-score ponderado:", f1_score(y_test, mlp_predictions, average='weighted'))
print("AUC (ponderado, one-vs-rest):", mlp_auc)

# dicionário de resultados
mlp_report = classification_report(y_test, mlp_predictions, output_dict=True)
results['MLP (Class_Weight)'] = {
    'accuracy': accuracy_score(y_test, mlp_predictions),
    'precision (macro)': mlp_report['macro avg']['precision'],
    'recall (macro)': mlp_report['macro avg']['recall'],
    'f1-score (macro)': mlp_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, mlp_predictions, average='weighted'),
    'auc (weighted_ovr)': mlp_auc
}

print("="*50)

Forma dos dados de treino (X_train, y_train): (19826, 5000) (19826,)
Forma dos dados de teste (X_test, y_test): (4957, 5000) (4957,)

Pesos das classes calculados (baseado em y_train):
{np.int64(0): np.float64(5.797076023391813), np.int64(1): np.float64(0.4303077657681122), np.int64(2): np.float64(1.9857772435897436)}

Treinando Regressão Logística com class_weight...
Avaliação da Regressão Logística:
              precision    recall  f1-score   support

           0       0.29      0.57      0.38       290
           1       0.97      0.84      0.90      3832
           2       0.72      0.93      0.81       835

    accuracy                           0.84      4957
   macro avg       0.66      0.78      0.70      4957
weighted avg       0.89      0.84      0.85      4957

Acurácia:  0.8376033891466613
F1-score ponderado: 0.8538630525491657
AUC (ponderado, one-vs-rest): 0.9313824779471941
--------------------------------------------------
Treinando Naive Bayes Multinomial (sem class_

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 30288
[LightGBM] [Info] Number of data points in the train set: 19826, number of used features: 979
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM:
              precision    recall  f1-score   support

           0       0.30      0.61      0.40       290
           1       0.97      0.85      0.90      3832
           2       0.76      0.94      0.84       835

    accuracy                           0.85      4957
   macro avg       0.68      0.80      0.72      4957
weighted avg       0.90      0.85      0.86      4957

Acurácia:  0.8484970748436554
F1-score ponderado: 0.8646944512588363
AUC (ponderado, one-vs-rest): 0.9386147953065481
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com class_weight...


...

 Total params: 3,940,619 (15.03 MB)

 Trainable params: 1,313,539 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,627,080 (10.02 MB)


Avaliando MLP no conjunto de teste original...

Acurácia da MLP no conjunto de teste original: 0.8318
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step

Avaliação completa da MLP:
              precision    recall  f1-score   support

           0       0.27      0.41      0.33       290
           1       0.93      0.87      0.90      3832
           2       0.73      0.80      0.76       835

    accuracy                           0.83      4957
   macro avg       0.64      0.69      0.66      4957
weighted avg       0.86      0.83      0.84      4957

F1-score ponderado: 0.8420082737857929
AUC (ponderado, one-vs-rest): 0.8977889468548846


In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np # para uso de unique e zip

# --- Pré-processamento para CNN ---

# O TfidfVectorizer não é ideal para CNNs, então opta-se por uma representação baseada em sequências/embeddings.

# 1. Tokenização para CNN (Sequências de Inteiros)
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_text'])

sequences = tokenizer.texts_to_sequences(df['processed_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded:")
print(padded_sequences[:5])
print("\nForma das sequências padded:", padded_sequences.shape)

# --- prepara labels para CNN ---
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(df['class'])

print("\nLabels originais:", df['class'].unique())
print("Labels codificadas:", encoded_y)

# --- divisão dos dados para CNN (usando as sequências e labels codificadas) ---
X_cnn = padded_sequences
y_cnn = encoded_y

X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.20, random_state=42)

print("\nForma dos dados de treino para CNN:", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de teste para CNN:", X_test_cnn.shape, y_test_cnn.shape)


# --- cálculo de Class Weights para a CNN (baseado em y_train_cnn) ---
classes_cnn = np.unique(y_train_cnn)
class_weights_cnn = compute_class_weight(class_weight='balanced', classes=classes_cnn, y=y_train_cnn)
class_weight_dict_cnn = dict(zip(classes_cnn, class_weights_cnn))

print("\nPesos das classes calculados para CNN (baseado em y_train_cnn):")
print(class_weight_dict_cnn)


# --- constrói o Modelo CNN ---
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- treinar o Modelo CNN (Com class_weight) ---
epochs = 10
batch_size = 32

print("\nTreinando CNN com class_weight...")
history = model.fit(X_train_cnn, y_train_cnn,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    verbose=1,
                    class_weight=class_weight_dict_cnn)

# --- avaliação do Modelo CNN ---
print("\nAvaliando CNN no conjunto de teste...")
loss, accuracy = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\nAcurácia da CNN no conjunto de teste: {accuracy:.4f}")

# classification report para CNN
cnn_predictions_probs = model.predict(X_test_cnn)
cnn_predictions = tf.argmax(cnn_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da CNN:")
print(classification_report(y_test_cnn, cnn_predictions))

# cálculo de F1-score ponderado e AUC para a CNN
cnn_f1_weighted = f1_score(y_test_cnn, cnn_predictions, average='weighted')

try:
    cnn_auc = roc_auc_score(y_test_cnn, cnn_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    cnn_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (CNN):", cnn_f1_weighted)
print("AUC (ponderado, one-vs-rest) (CNN):", cnn_auc)

# dicionário de resultados
try:
    cnn_report = classification_report(y_test_cnn, cnn_predictions, output_dict=True)
    results['CNN (Embedding+Seq, Class_Weight)'] = { # Alterar a chave para refletir class_weight
        'accuracy': accuracy_score(y_test_cnn, cnn_predictions),
        'precision (macro)': cnn_report['macro avg']['precision'],
        'recall (macro)': cnn_report['macro avg']['recall'],
        'f1-score (macro)': cnn_report['macro avg']['f1-score'],
        'f1-score (weighted)': cnn_f1_weighted,
        'auc (weighted_ovr)': cnn_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da CNN (com class_weight) não foram armazenados.")


Exemplo de sequências padded:
[[   3  100  838  920 2932  204   20   38   84   71   17    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3   96   78 8148 5799   37 2045   78    4  457  381    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando CNN com class_weight...
Epoch 1/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.7128 - loss: 0.7976 - val_accuracy: 0.8048 - val_loss: 0.5176
Epoch 2/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.8367 - loss: 0.3801 - val_accuracy: 0.8522 - val_loss: 0.3964
Epoch 3/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9171 - loss: 0.1943 - val_accuracy: 0.8341 - val_loss: 0.4672
Epoch 4/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 23s 42ms/step - accuracy: 0.9540 - loss: 0.0988 - val_accuracy: 0.8608 - val_loss: 0.5239
Epoch 5/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 42s 44ms/step - accuracy: 0.9729 - loss: 0.0600 - val_accuracy: 0.8724 - val_loss: 0.5337
Epoch 6/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.9810 - loss: 0.0460 - val_accuracy: 0.8754 - val_loss: 0.5449
Epoch 7/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 44s 49ms/step - accuracy: 0.9817 - loss: 0.0397 - val_accuracy: 0.8659 - val_loss: 0.5881
Epoch 8/10
558/558 ━━━━━━━━━━━━━━━━━━━━ 25s 45ms/step -

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# --- reutiliza dados preparados para a CNN --
# --- cálculo de Class Weights para a LSTM (baseado em y_train_cnn) ---
classes_lstm = np.unique(y_train_cnn)
class_weights_lstm = compute_class_weight(class_weight='balanced', classes=classes_lstm, y=y_train_cnn)
class_weight_dict_lstm = dict(zip(classes_lstm, class_weights_lstm))

print("\nPesos das classes calculados para LSTM (baseado em y_train_cnn):")
print(class_weight_dict_lstm)

# --- constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

model_lstm = Sequential([
    # camada de Embedding: reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # camada LSTM
    LSTM(128), #

    # Camada de Dropout
    Dropout(0.5),

    # Camada Densa (Fully Connected)
    Dense(64, activation='relu'), # Camada densa adicional
    Dropout(0.5),

    # Camada de saída (igual à CNN)
    Dense(len(label_encoder.classes_), activation='softmax')
])


model_lstm.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model_lstm.summary()

# --- treino o Modelo LSTM (Com class_weight) ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM com class_weight...")
history_lstm = model_lstm.fit(X_train_cnn, y_train_cnn,
                              epochs=epochs_lstm,
                              batch_size=batch_size_lstm,
                              validation_split=0.1,
                              verbose=1,
                              class_weight=class_weight_dict_lstm)


# --- avaliação do Modelo LSTM ---
print("\nAvaliando LSTM no conjunto de teste...")
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\nAcurácia da LSTM no conjunto de teste: {accuracy_lstm:.4f}")

# classification report para LSTM
lstm_predictions_probs = model_lstm.predict(X_test_cnn)
lstm_predictions = tf.argmax(lstm_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da LSTM:")

class_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test_cnn, lstm_predictions, target_names=class_names))

# cálculo de F1-score ponderado e AUC para a LSTM
lstm_f1_weighted = f1_score(y_test_cnn, lstm_predictions, average='weighted')

try:
    lstm_auc = roc_auc_score(y_test_cnn, lstm_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    lstm_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (LSTM):", lstm_f1_weighted)
print("AUC (ponderado, one-vs-rest) (LSTM):", lstm_auc)


print("="*50)

# dicionário de resultados
try:
    lstm_report = classification_report(y_test_cnn, lstm_predictions, output_dict=True, target_names=class_names)
    results['LSTM (Embedding+Seq, Class_Weight)'] = {
        'accuracy': accuracy_score(y_test_cnn, lstm_predictions),
        'precision (macro)': lstm_report['macro avg']['precision'],
        'recall (macro)': lstm_report['macro avg']['recall'],
        'f1-score (macro)': lstm_report['macro avg']['f1-score'],
        'f1-score (weighted)': lstm_f1_weighted,
        'auc (weighted_ovr)': lstm_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da LSTM (com class_weight) não foram armazenados.")


Pesos das classes calculados para LSTM (baseado em y_train_cnn):
{np.int64(0): np.float64(5.797076023391813), np.int64(1): np.float64(0.4303077657681122), np.int64(2): np.float64(1.9857772435897436)}

Construindo o Modelo LSTM...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando LSTM com class_weight...
Epoch 1/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 132s 227ms/step - accuracy: 0.3139 - loss: 1.1076 - val_accuracy: 0.0514 - val_loss: 1.1129
Epoch 2/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 137s 218ms/step - accuracy: 0.2049 - loss: 1.1206 - val_accuracy: 0.7796 - val_loss: 1.0928
Epoch 3/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 137s 208ms/step - accuracy: 0.2231 - loss: 1.1090 - val_accuracy: 0.7796 - val_loss: 1.0893
Epoch 4/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 170s 259ms/step - accuracy: 0.2071 - loss: 1.1169 - val_accuracy: 0.7796 - val_loss: 1.0520
Epoch 5/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 173s 206ms/step - accuracy: 0.3215 - loss: 1.1044 - val_accuracy: 0.1689 - val_loss: 1.0845
Epoch 6/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 112s 201ms/step - accuracy: 0.2629 - loss: 1.1149 - val_accuracy: 0.0514 - val_loss: 1.0991
Epoch 7/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 114s 204ms/step - accuracy: 0.0917 - loss: 1.1275 - val_accuracy: 0.1689 - val_loss: 1.0676
Epoch 8/15
558/558 ━━━━━━━━━━━━━━━━━━━━ 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

#### **Oversampling**

Como segunda abordagem para tratar o desbalanceamento entre as classes, foi utilizado o método de oversampling sintético **SMOTE (Synthetic Minority Over-sampling Technique)**. Essa técnica atua gerando novos exemplos sintéticos para as classes minoritárias com base na interpolação entre amostras reais vizinhas no espaço de atributos, equilibrando a distribuição sem simplesmente duplicar instâncias existentes.

O SMOTE foi aplicado exclusivamente ao conjunto de treino, preservando a distribuição real nos dados de teste. Com isso, as três classes (0, 1 e 2) passaram a ter exatamente o mesmo número de instâncias (15.358), conforme evidenciado após o reamostramento.

Esse balanceamento favorece o aprendizado de modelos como Regressão Logística, SVM, Random Forest, LightGBM e MLP, aumentando sua capacidade de generalização para exemplos das classes originalmente sub-representadas. A técnica é especialmente vantajosa por evitar overfitting comum em métodos de duplicação, ao mesmo tempo em que permite o uso direto dos classificadores.

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [ ]:
print("\nDistribuição das classes no treino (antes do balanceamento):")
print(y_train.value_counts())

# inicialiaz SMOTE
# random_state para reprodutibilidade
# sampling_strategy='auto' reamostra todas as classes minoritárias para igualar a classe majoritária
smote = SMOTE(sampling_strategy='auto', random_state=42)

# aplica SMOTE apenas nos dados de TREINO
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("\nForma dos dados de treino após SMOTE:", X_train_resampled.shape, y_train_resampled.shape)
print("\nDistribuição das classes no treino (depois do SMOTE):")
print(y_train_resampled.value_counts())

# uso do X_train_resampled e y_train_resampled para treinar modelos TF-IDF
# (Logistic Regression, Naive Bayes, SVM, Random Forest, LightGBM, MLP)


Distribuição das classes no treino (antes do balanceamento):
class
1    15358
2     3328
0     1140
Name: count, dtype: int64

Forma dos dados de treino após SMOTE: (46074, 5000) (46074,)

Distribuição das classes no treino (depois do SMOTE):
class
0    15358
2    15358
1    15358
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import pandas as pd

results = {}
# considera X_train_resampled, y_train_resampled, X_test, y_test já estão definidos

# 1. Regressão Logística
print("Treinando Regressão Logística com SMOTE-balanced data...")
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_resampled, y_train_resampled)
lr_predictions = lr_model.predict(X_test)

print("Avaliação da Regressão Logística:")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))

# cálculo de F1-score ponderado
lr_f1_weighted = f1_score(y_test, lr_predictions, average='weighted')
print("F1-score ponderado:", lr_f1_weighted)

# cálculo de AUC
try:
    lr_probabilities = lr_model.predict_proba(X_test)
    lr_auc = roc_auc_score(y_test, lr_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lr_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", lr_auc)

# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': lr_f1_weighted,
    'auc (weighted_ovr)': lr_auc
}


print("-" * 50)

# 2. Naive Bayes Multinomial (geralmente bom para dados textuais esparsos como TF-IDF)
# Naive Bayes pode ser sensível ao oversampling sintético como SMOTE,
# mas o treino será feito com os dados reamostrados para comparação.

print("Treinando Naive Bayes Multinomial com SMOTE-balanced data...")
nb_model = MultinomialNB()
nb_model.fit(X_train_resampled, y_train_resampled)
nb_predictions = nb_model.predict(X_test)

print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))

# cálculo de F1-score ponderado
nb_f1_weighted = f1_score(y_test, nb_predictions, average='weighted')
print("F1-score ponderado:", nb_f1_weighted)

# cálculo de AUC
try:
    nb_probabilities = nb_model.predict_proba(X_test)
    nb_auc = roc_auc_score(y_test, nb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    nb_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", nb_auc)


# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': nb_f1_weighted,
    'auc (weighted_ovr)': nb_auc
}

print("-" * 50)

# 3. Support Vector Machine (SVM)
print("Treinando SVM (Kernel Linear) com SMOTE-balanced data...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_resampled, y_train_resampled)
svm_predictions = svm_model.predict(X_test)

print("Avaliação do SVM:")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))

# cálculo de F1-score ponderado
svm_f1_weighted = f1_score(y_test, svm_predictions, average='weighted')
print("F1-score ponderado:", svm_f1_weighted)

# cálculo de AUC
try:
    if hasattr(svm_model, 'predict_proba'):
        svm_probabilities = svm_model.predict_proba(X_test)
        svm_auc = roc_auc_score(y_test, svm_probabilities, multi_class='ovr', average='weighted')
    else:
        svm_auc = "N/A (probability=False)"
except AttributeError:
    svm_auc = "N/A (problema ao obter predict_proba)"
print("AUC (ponderado, one-vs-rest):", svm_auc)


# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': svm_f1_weighted,
    'auc (weighted_ovr)': svm_auc
}

print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest com SMOTE-balanced data...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # número de árvores
rf_model.fit(X_train_resampled, y_train_resampled)
rf_predictions = rf_model.predict(X_test)

print("Avaliação do Random Forest:")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))

# cálculo de F1-score ponderado
rf_f1_weighted = f1_score(y_test, rf_predictions, average='weighted')
print("F1-score ponderado:", rf_f1_weighted)

# cálculo de AUC
try:
    rf_probabilities = rf_model.predict_proba(X_test)
    rf_auc = roc_auc_score(y_test, rf_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    rf_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", rf_auc)


# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': rf_f1_weighted,
    'auc (weighted_ovr)': rf_auc
}

Treinando Regressão Logística com SMOTE-balanced data...
Avaliação da Regressão Logística:
              precision    recall  f1-score   support

           0       0.27      0.56      0.36       290
           1       0.96      0.85      0.90      3832
           2       0.77      0.89      0.83       835

    accuracy                           0.84      4957
   macro avg       0.67      0.77      0.70      4957
weighted avg       0.89      0.84      0.86      4957

Acurácia:  0.8396207383498083
F1-score ponderado: 0.8572308795413649
AUC (ponderado, one-vs-rest): 0.9265650506968165
--------------------------------------------------
Treinando Naive Bayes Multinomial com SMOTE-balanced data...
Avaliação do Naive Bayes Multinomial:
              precision    recall  f1-score   support

           0       0.23      0.57      0.32       290
           1       0.94      0.81      0.87      3832
           2       0.71      0.77      0.74       835

    accuracy                           0.7

In [ ]:
# --- Modelos Avançados com TF-IDF ---

# 5. LightGBM (Gradient Boosting)

import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

print("\n" + "="*50)
print("Treinando LightGBM com SMOTE-balanced data...")

# converte matriz esparsa para o formato LightGBM
# LightGBM funciona diretamente com matrizes esparsas do SciPy
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=len(df['class'].unique()), random_state=42)
lgb_model.fit(X_train_resampled, y_train_resampled)
lgb_predictions = lgb_model.predict(X_test)

print("Avaliação do LightGBM:")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))

# cálculo de F1-score ponderado
lgb_f1_weighted = f1_score(y_test, lgb_predictions, average='weighted')
print("F1-score ponderado:", lgb_f1_weighted)

# cálculo de AUC
try:
    lgb_probabilities = lgb_model.predict_proba(X_test)
    lgb_auc = roc_auc_score(y_test, lgb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lgb_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", lgb_auc)


# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': lgb_f1_weighted,
    'auc (weighted_ovr)': lgb_auc
}


print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


print("Treinando Rede Neural Densa (MLP) com SMOTE-balanced data...")

# A matriz TF-IDF balanceada já está pronta (X_train_resampled, y_train_resampled)
# O X_test e y_test originais são usados para avaliação

# constrói o Modelo MLP
mlp_model = Sequential([
    # Flatten é necessário se a entrada não for 1D, mas a matriz TF-IDF já é 2D (samples x features)
    Dense(256, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # camada de saída (igual à CNN para classificação multiclasse)
    Dense(len(df['class'].unique()), activation='softmax')
])

mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

mlp_model.summary()


epochs_mlp = 20
batch_size_mlp = 64

print("\nTreinando MLP com dados SMOTE...")
history_mlp = mlp_model.fit(X_train_resampled, y_train_resampled,
                          epochs=epochs_mlp,
                          batch_size=batch_size_mlp,
                          validation_split=0.1, # validar no conjunto de treino reamostrado
                          verbose=1)


# avaliação do Modelo MLP
print("\nAvaliando MLP (SMOTE) no conjunto de teste original...")
loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP (SMOTE) no conjunto de teste original: {accuracy_mlp:.4f}")

# classification report para MLP (SMOTE)
mlp_predictions_probs = mlp_model.predict(X_test) # prediz no teste original
mlp_predictions = tf.argmax(mlp_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da MLP (SMOTE):")
print(classification_report(y_test, mlp_predictions))

# cálculo de F1-score ponderado
mlp_f1_weighted = f1_score(y_test, mlp_predictions, average='weighted')
print("F1-score ponderado:", mlp_f1_weighted)

# cálculo de AUC
try:
    mlp_auc = roc_auc_score(y_test, mlp_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    mlp_auc = f"N/A (problema ao calcular AUC: {e})"
print("AUC (ponderado, one-vs-rest):", mlp_auc)


# dicionário de resultados
try:
    mlp_report = classification_report(y_test, mlp_predictions, output_dict=True)
    results['MLP (SMOTE)'] = {
        'accuracy': accuracy_score(y_test, mlp_predictions),
        'precision (macro)': mlp_report['macro avg']['precision'],
        'recall (macro)': mlp_report['macro avg']['recall'],
        'f1-score (macro)': mlp_report['macro avg']['f1-score'],
        'f1-score (weighted)': mlp_f1_weighted,
        'auc (weighted_ovr)': mlp_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da MLP (SMOTE) não foram armazenados.")


print("="*50)


Treinando LightGBM com SMOTE-balanced data...


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.389420 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 81147
[LightGBM] [Info] Number of data points in the train set: 46074, number of used features: 2338
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM:
              precision    recall  f1-score   support

           0       0.35      0.53      0.42       290
           1       0.96      0.88      0.92      3832
           2       0.78      0.94      0.85       835

    accuracy                           0.87      4957
   macro avg       0.70      0.79      0.73      4957
weighted avg       0.90      0.87      0.88      4957

Acurácia:  0.8723017954407908
F1-score ponderado: 0.8811002395679967
AUC (ponderado, one-vs-rest): 0.9388846494445826
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com SMOTE-balanced data...


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                │ (None, 256)            │     1,280,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,539 (5.01 MB)

 Trainable params: 1,313,539 (5.01 MB)

 Non-trainable params: 0 (0.00 B)


Treinando MLP com dados SMOTE...
Epoch 1/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - accuracy: 0.7330 - loss: 0.6325 - val_accuracy: 0.9811 - val_loss: 0.1209
Epoch 2/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.9538 - loss: 0.1480 - val_accuracy: 0.9848 - val_loss: 0.0877
Epoch 3/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.9767 - loss: 0.0725 - val_accuracy: 0.9939 - val_loss: 0.0342
Epoch 4/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.9847 - loss: 0.0503 - val_accuracy: 0.9974 - val_loss: 0.0293
Epoch 5/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.9871 - loss: 0.0400 - val_accuracy: 0.9948 - val_loss: 0.0242
Epoch 6/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.9898 - loss: 0.0318 - val_accuracy: 0.9976 - val_loss: 0.0211
Epoch 7/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.9910 - loss: 0.0289 - val_accuracy: 0.9813 - val_loss: 0.0373
Epoch 8/20
648/648 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - 

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

# --- Pré-processamento para CNN ---

# O TfidfVectorizer não é ideal para CNNs, precisamos de uma representação baseada em sequências/embeddings.

# 1. Tokenização para CNN (Sequências de Inteiros)
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_text'])

sequences = tokenizer.texts_to_sequences(df['processed_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded:")
print(padded_sequences[:5])
print("\nForma das sequências padded:", padded_sequences.shape)

# --- prepara labels para CNN ---
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(df['class'])

print("\nLabels originais:", df['class'].unique())
print("Labels codificadas:", encoded_y)

# --- divide dados para CNN (usando as sequências e labels codificadas) ---
X_cnn = padded_sequences
y_cnn = encoded_y

# divisão original em treino e teste para a CNN
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.20, random_state=42)

print("\nForma dos dados de treino para CNN (antes do SMOTE):", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de teste para CNN:", X_test_cnn.shape, y_test_cnn.shape)
print("\nDistribuição das classes no treino da CNN (antes do balanceamento):")
print(pd.Series(y_train_cnn).value_counts())


# --- aplica SMOTE nos dados de TREINO da CNN ---
try:
    smote_cnn = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_cnn_resampled_smote, y_train_cnn_resampled_smote = smote_cnn.fit_resample(X_train_cnn, y_train_cnn)

    print("\nForma dos dados de treino da CNN após SMOTE:", X_train_cnn_resampled_smote.shape, y_train_cnn_resampled_smote.shape)
    print("\nDistribuição das classes no treino da CNN (depois do SMOTE):")
    print(pd.Series(y_train_cnn_resampled_smote).value_counts())

except NameError:
    print("Erro: imblearn.over_sampling.SMOTE não encontrado. Certifique-se de que imbalanced-learn está instalado e SMOTE foi importado.")
    raise # relança erro pra interromper execução sem SMOTE


# --- contrói o Modelo CNN ---
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

model_cnn_smote = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn_smote.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_cnn_smote.summary()

# --- treino do Modelo CNN (com dados Oversampled SMOTE) ---
epochs = 10
batch_size = 32

print("\nTreinando CNN com dados Oversampled (SMOTE)...")
history_cnn_smote = model_cnn_smote.fit(X_train_cnn_resampled_smote, y_train_cnn_resampled_smote,
                                        epochs=epochs,
                                        batch_size=batch_size,
                                        validation_split=0.1,
                                        verbose=1)


# --- avaliação do modelo
print("\nAvaliando CNN (SMOTE) no conjunto de teste original...")
loss_cnn_smote, accuracy_cnn_smote = model_cnn_smote.evaluate(X_test_cnn, y_test_cnn, verbose=0)

print(f"\nAcurácia da CNN (SMOTE) no conjunto de teste original: {accuracy_cnn_smote:.4f}")

# classification report para CNN (SMOTE)
cnn_smote_predictions_probs = model_cnn_smote.predict(X_test_cnn)
cnn_smote_predictions = tf.argmax(cnn_smote_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da CNN (SMOTE):")
print(classification_report(y_test_cnn, cnn_smote_predictions))

# cálculo do F1-score ponderado e AUC para a CNN (SMOTE)
cnn_smote_f1_weighted = f1_score(y_test_cnn, cnn_smote_predictions, average='weighted')

try:
    cnn_smote_auc = roc_auc_score(y_test_cnn, cnn_smote_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    cnn_smote_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (CNN SMOTE):", cnn_smote_f1_weighted)
print("AUC (ponderado, one-vs-rest) (CNN SMOTE):", cnn_smote_auc)

# dicionário de resultados
try:
    cnn_smote_report = classification_report(y_test_cnn, cnn_smote_predictions, output_dict=True)
    results['CNN (Embedding+Seq, SMOTE)'] = {
        'accuracy': accuracy_score(y_test_cnn, cnn_smote_predictions),
        'precision (macro)': cnn_smote_report['macro avg']['precision'],
        'recall (macro)': cnn_smote_report['macro avg']['recall'],
        'f1-score (macro)': cnn_smote_report['macro avg']['f1-score'],
        'f1-score (weighted)': cnn_smote_f1_weighted,
        'auc (weighted_ovr)': cnn_smote_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da CNN (SMOTE) não foram armazenados.")


Exemplo de sequências padded:
[[   3  100  838  920 2932  204   20   38   84   71   17    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3   96   78 8148 5799   37 2045   78    4  457  381    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_6          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando CNN com dados Oversampled (SMOTE)...
Epoch 1/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 58s 43ms/step - accuracy: 0.6130 - loss: 0.8303 - val_accuracy: 0.3444 - val_loss: 1.1217
Epoch 2/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 56s 43ms/step - accuracy: 0.7785 - loss: 0.5448 - val_accuracy: 0.3333 - val_loss: 1.2339
Epoch 3/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 56s 43ms/step - accuracy: 0.8668 - loss: 0.3463 - val_accuracy: 0.3576 - val_loss: 1.3700
Epoch 4/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 55s 43ms/step - accuracy: 0.9287 - loss: 0.1985 - val_accuracy: 0.3885 - val_loss: 1.8538
Epoch 5/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 82s 43ms/step - accuracy: 0.9598 - loss: 0.1212 - val_accuracy: 0.4377 - val_loss: 1.9127
Epoch 6/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - accuracy: 0.9713 - loss: 0.0820 - val_accuracy: 0.4015 - val_loss: 2.9573
Epoch 7/10
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 80s 43ms/step - accuracy: 0.9790 - loss: 0.0613 - val_accuracy: 0.4974 - val_loss: 2.5173
Epoch 8/10
1296/1296 ━━━━━━━

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# --- reutiliza dados preparados para a CNN ---

# --- Aplicar SMOTE nos dados de TREINO da LSTM (que são os mesmos da CNN) ---

print("\nDistribuição das classes no treino da LSTM (antes do balanceamento):")
print(pd.Series(y_train_cnn).value_counts())

try:
    smote_lstm = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_lstm_resampled_smote, y_train_lstm_resampled_smote = smote_lstm.fit_resample(X_train_cnn, y_train_cnn)

    print("\nForma dos dados de treino da LSTM após SMOTE:", X_train_lstm_resampled_smote.shape, y_train_lstm_resampled_smote.shape)
    print("\nDistribuição das classes no treino da LSTM (depois do SMOTE):")
    print(pd.Series(y_train_lstm_resampled_smote).value_counts())

except NameError:
    print("Erro: imblearn.over_sampling.SMOTE não encontrado. Certifique-se de que imbalanced-learn está instalado e SMOTE foi importado.")
    raise # relança o erro para interromper execução sem SMOTE


# --- constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

model_lstm_smote = Sequential([
    # camada de Embedding: reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # camada LSTM
    LSTM(128),

    # camada de Dropout
    Dropout(0.5),

    # camada Densa (Fully Connected)
    Dense(64, activation='relu'), # Camada densa adicional
    Dropout(0.5),

    # camada de saída (igual à CNN)
    Dense(len(label_encoder.classes_), activation='softmax')
])


model_lstm_smote.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model_lstm_smote.summary()

# --- treino do Modelo LSTM (com dados Oversampled SMOTE) ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM com dados Oversampled (SMOTE)...")
history_lstm_smote = model_lstm_smote.fit(X_train_lstm_resampled_smote, y_train_lstm_resampled_smote,
                                          epochs=epochs_lstm,
                                          batch_size=batch_size_lstm,
                                          validation_split=0.1, # validação nos dados reamostrados
                                          verbose=1)


# --- avaliação do Modelo LSTM ---
print("\nAvaliando LSTM (SMOTE) no conjunto de teste original...")
loss_lstm_smote, accuracy_lstm_smote = model_lstm_smote.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f"\nAcurácia da LSTM (SMOTE) no conjunto de teste original: {accuracy_lstm_smote:.4f}")

# classification report para LSTM (SMOTE)
lstm_smote_predictions_probs = model_lstm_smote.predict(X_test_cnn)
lstm_smote_predictions = tf.argmax(lstm_smote_predictions_probs, axis=1).numpy()
print("\nAvaliação completa da LSTM (SMOTE):")


class_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test_cnn, lstm_smote_predictions, target_names=class_names))

# cálculo do F1-score ponderado e AUC para a LSTM (SMOTE)
lstm_smote_f1_weighted = f1_score(y_test_cnn, lstm_smote_predictions, average='weighted')

try:
    lstm_smote_auc = roc_auc_score(y_test_cnn, lstm_smote_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    lstm_smote_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (LSTM SMOTE):", lstm_smote_f1_weighted)
print("AUC (ponderado, one-vs-rest) (LSTM SMOTE):", lstm_smote_auc)


print("="*50)

# dicionário de resultados
try:
    lstm_smote_report = classification_report(y_test_cnn, lstm_smote_predictions, output_dict=True, target_names=class_names)
    results['LSTM (Embedding+Seq, SMOTE)'] = {
        'accuracy': accuracy_score(y_test_cnn, lstm_smote_predictions),
        'precision (macro)': lstm_smote_report['macro avg']['precision'],
        'recall (macro)': lstm_smote_report['macro avg']['recall'],
        'f1-score (macro)': lstm_smote_report['macro avg']['f1-score'],
        'f1-score (weighted)': lstm_smote_f1_weighted,
        'auc (weighted_ovr)': lstm_smote_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da LSTM (SMOTE) não foram armazenados.")


Distribuição das classes no treino da LSTM (antes do balanceamento):
1    15358
2     3328
0     1140
Name: count, dtype: int64

Forma dos dados de treino da LSTM após SMOTE: (46074, 100) (46074,)

Distribuição das classes no treino da LSTM (depois do SMOTE):
0    15358
2    15358
1    15358
Name: count, dtype: int64

Construindo o Modelo LSTM...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando LSTM com dados Oversampled (SMOTE)...
Epoch 1/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 261s 198ms/step - accuracy: 0.3681 - loss: 1.0895 - val_accuracy: 0.0000e+00 - val_loss: 1.2968
Epoch 2/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 259s 200ms/step - accuracy: 0.3665 - loss: 1.0865 - val_accuracy: 0.0000e+00 - val_loss: 1.3008
Epoch 3/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 258s 199ms/step - accuracy: 0.3657 - loss: 1.0856 - val_accuracy: 0.0000e+00 - val_loss: 1.3554
Epoch 4/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 260s 201ms/step - accuracy: 0.3693 - loss: 1.0863 - val_accuracy: 0.0000e+00 - val_loss: 1.3699
Epoch 5/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 260s 199ms/step - accuracy: 0.3682 - loss: 1.0842 - val_accuracy: 0.0000e+00 - val_loss: 1.3306
Epoch 6/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 261s 201ms/step - accuracy: 0.3730 - loss: 1.0857 - val_accuracy: 0.0000e+00 - val_loss: 1.3176
Epoch 7/15
1296/1296 ━━━━━━━━━━━━━━━━━━━━ 260s 200ms/step - accuracy: 0.3663 - loss: 1.0867 - val_accuracy: 0.0000e+00 - va

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

#### **Undersampling**

Como terceira estratégia de balanceamento, foi empregado o método de undersampling aleatório, utilizando a técnica **RandomUnderSampler** da biblioteca `imbalanced-learn`. Diferente do SMOTE, que gera novas amostras para as classes minoritárias, o undersampling reduz o número de instâncias da classe majoritária, removendo exemplos aleatórios até que todas as classes tenham o mesmo número de ocorrências.

No experimento, o balanceamento foi aplicado apenas sobre o conjunto de treino, **igualando as classes 0, 1 e 2 com 1.140 exemplos cada** — valor correspondente à classe originalmente menos representada. Essa abordagem visa mitigar o viés dos modelos em favor da classe dominante, porém com o custo potencial de perda de informação relevante.
Apesar disso, ela permite a avaliação do impacto da redução de dados no desempenho de algoritmos como **Regressão Logística, Naive Bayes, SVM, Random Forest, LightGBM e redes neurais densas**, todos treinados com a base reamostrada, possibilitando uma comparação direta com os resultados obtidos nas estratégias anteriores de balanceamento.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

print("Forma dos dados de treino (X_train, y_train) antes do undersampling:", X_train.shape, y_train.shape)
print("Forma dos dados de teste (X_test, y_test):", X_test.shape, y_test.shape)
print("\nDistribuição das classes no treino (antes do undersampling):")
print(y_train.value_counts())


# --- aplica undersampling aleatório nos dados de TREINO ---
try:
    # inicializa RandomUnderSampler
    # sampling_strategy='auto' remove instâncias da classe majoritária para igualar a classe minoritária
    undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

    X_train_resampled_under, y_train_resampled_under = undersampler.fit_resample(X_train, y_train)

    print("\nForma dos dados de treino após Undersampling:", X_train_resampled_under.shape, y_train_resampled_under.shape)
    print("\nDistribuição das classes no treino (depois do Undersampling):")
    print(pd.Series(y_train_resampled_under).value_counts())

except NameError:
    print("Erro: imblearn.under_sampling.RandomUnderSampler não encontrado. Certifique-se de que imbalanced-learn está instalado e RandomUnderSampler foi importado.").
    raise # relança o erro

Forma dos dados de treino (X_train, y_train) antes do undersampling: (19826, 5000) (19826,)
Forma dos dados de teste (X_test, y_test): (4957, 5000) (4957,)

Distribuição das classes no treino (antes do undersampling):
class
1    15358
2     3328
0     1140
Name: count, dtype: int64

Forma dos dados de treino após Undersampling: (3420, 5000) (3420,)

Distribuição das classes no treino (depois do Undersampling):
class
0    1140
1    1140
2    1140
Name: count, dtype: int64


In [ ]:
# inicializa dicionário
# results = {}

# 1. Regressão Logística
print("\n" + "="*50)
print("Treinando Regressão Logística com Undersampled data...")
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_resampled_under, y_train_resampled_under)
lr_predictions = lr_model.predict(X_test)

print("Avaliação da Regressão Logística:")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))

# cálculo de F1-score ponderado
lr_f1_weighted = f1_score(y_test, lr_predictions, average='weighted')
print("F1-score ponderado:", lr_f1_weighted)

# cálculo de AUC
try:
    lr_probabilities = lr_model.predict_proba(X_test)
    lr_auc = roc_auc_score(y_test, lr_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lr_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", lr_auc)

# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': lr_f1_weighted,
    'auc (weighted_ovr)': lr_auc
}

print("-" * 50)

# 2. Naive Bayes Multinomial
print("Treinando Naive Bayes Multinomial com Undersampled data...")
nb_model = MultinomialNB()
nb_model.fit(X_train_resampled_under, y_train_resampled_under)
nb_predictions = nb_model.predict(X_test)

print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))

# cálculo de F1-score ponderado
nb_f1_weighted = f1_score(y_test, nb_predictions, average='weighted')
print("F1-score ponderado:", nb_f1_weighted)

# cálculo de AUC
try:
    nb_probabilities = nb_model.predict_proba(X_test)
    nb_auc = roc_auc_score(y_test, nb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    nb_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", nb_auc)

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': nb_f1_weighted,
    'auc (weighted_ovr)': nb_auc
}

print("-" * 50)

# 3. Support Vector Machine (SVM)
print("Treinando SVM (Kernel Linear) com Undersampled data...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_resampled_under, y_train_resampled_under)
svm_predictions = svm_model.predict(X_test)

print("Avaliação do SVM:")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))

# cálculo de F1-score ponderado
svm_f1_weighted = f1_score(y_test, svm_predictions, average='weighted')
print("F1-score ponderado:", svm_f1_weighted)

# cálculo de AUC
try:
    if hasattr(svm_model, 'predict_proba'):
        svm_probabilities = svm_model.predict_proba(X_test)
        svm_auc = roc_auc_score(y_test, svm_probabilities, multi_class='ovr', average='weighted')
    else:
        svm_auc = "N/A (probability=False)"
except AttributeError:
    svm_auc = "N/A (problema ao obter predict_proba)"
print("AUC (ponderado, one-vs-rest):", svm_auc)

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': svm_f1_weighted,
    'auc (weighted_ovr)': svm_auc
}

print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest com Undersampled data...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42) # número de árvores
rf_model.fit(X_train_resampled_under, y_train_resampled_under)
rf_predictions = rf_model.predict(X_test)

print("Avaliação do Random Forest:")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))

# cálculo de F1-score ponderado
rf_f1_weighted = f1_score(y_test, rf_predictions, average='weighted')
print("F1-score ponderado:", rf_f1_weighted)

# cálculo de AUC
try:
    rf_probabilities = rf_model.predict_proba(X_test)
    rf_auc = roc_auc_score(y_test, rf_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    rf_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", rf_auc)

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': rf_f1_weighted,
    'auc (weighted_ovr)': rf_auc
}

print("-" * 50)


Treinando Regressão Logística com Undersampled data...
Avaliação da Regressão Logística:
              precision    recall  f1-score   support

           0       0.24      0.64      0.35       290
           1       0.97      0.78      0.86      3832
           2       0.69      0.93      0.79       835

    accuracy                           0.80      4957
   macro avg       0.64      0.78      0.67      4957
weighted avg       0.88      0.80      0.82      4957

Acurácia:  0.795239055880573
F1-score ponderado: 0.8222035141956675
AUC (ponderado, one-vs-rest): 0.9176774330625891
--------------------------------------------------
Treinando Naive Bayes Multinomial com Undersampled data...
Avaliação do Naive Bayes Multinomial:
              precision    recall  f1-score   support

           0       0.18      0.72      0.29       290
           1       0.94      0.74      0.83      3832
           2       0.77      0.71      0.74       835

    accuracy                           0.73   

In [ ]:
# 5. LightGBM (Gradient Boosting)

print("Treinando LightGBM com Undersampled data...")

lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=len(df['class'].unique()), random_state=42)
lgb_model.fit(X_train_resampled_under, y_train_resampled_under)
lgb_predictions = lgb_model.predict(X_test)

print("Avaliação do LightGBM:")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))

# cálculo de F1-score ponderado
lgb_f1_weighted = f1_score(y_test, lgb_predictions, average='weighted')
print("F1-score ponderado:", lgb_f1_weighted)

# cálculo de AUC
try:
    lgb_probabilities = lgb_model.predict_proba(X_test)
    lgb_auc = roc_auc_score(y_test, lgb_probabilities, multi_class='ovr', average='weighted')
except AttributeError:
    lgb_auc = "N/A (predict_proba não disponível ou com problema)"
print("AUC (ponderado, one-vs-rest):", lgb_auc)

# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': lgb_f1_weighted,
    'auc (weighted_ovr)': lgb_auc
}

print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras
print("Treinando Rede Neural Densa (MLP) com Undersampled data...")


mlp_model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_resampled_under.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(df['class'].unique()), activation='softmax')
])

mlp_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

mlp_model.summary()

# treino do Modelo MLP
epochs_mlp = 20
batch_size_mlp = 64

print("\nTreinando MLP com dados Undersampled...")

history_mlp = mlp_model.fit(X_train_resampled_under, y_train_resampled_under,
                          epochs=epochs_mlp,
                          batch_size=batch_size_mlp,
                          validation_split=0.1,
                          verbose=1)


# avaliação do Modelo MLP
print("\nAvaliando MLP (Undersampling) no conjunto de teste original...")
loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP (Undersampling) no conjunto de teste original: {accuracy_mlp:.4f}")

# classification report para MLP
mlp_predictions_probs = mlp_model.predict(X_test)
mlp_predictions = tf.argmax(mlp_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da MLP (Undersampling):")
print(classification_report(y_test, mlp_predictions))

# cálculo de F1-score ponderado
mlp_f1_weighted = f1_score(y_test, mlp_predictions, average='weighted')
print("F1-score ponderado:", mlp_f1_weighted)

# cálculo de AUC
try:
    mlp_auc = roc_auc_score(y_test, mlp_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    mlp_auc = f"N/A (problema ao calcular AUC: {e})"
print("AUC (ponderado, one-vs-rest):", mlp_auc)

# dicionário de resultados
mlp_report = classification_report(y_test, mlp_predictions, output_dict=True)
results['MLP (Undersampling)'] = {
    'accuracy': accuracy_score(y_test, mlp_predictions),
    'precision (macro)': mlp_report['macro avg']['precision'],
    'recall (macro)': mlp_report['macro avg']['recall'],
    'f1-score (macro)': mlp_report['macro avg']['f1-score'],
    'f1-score (weighted)': mlp_f1_weighted,
    'auc (weighted_ovr)': mlp_auc
}

print("="*50)

Treinando LightGBM com Undersampled data...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4141
[LightGBM] [Info] Number of data points in the train set: 3420, number of used features: 193
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM:
              precision    recall  f1-score   support

           0       0.20      0.65      0.31       290
           1       0.97      0.76      0.85      3832
           2       0.73      0.90      0.81       835

    accuracy                           0.78      4957
   macro avg       0.64      0.77      0.66      4957
weighted avg       0.89      0.78      0.81      4957

Acurácia:  0.7778898527335082
F1-score ponderado: 0.8141790112878247


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


AUC (ponderado, one-vs-rest): 0.9211633741075974
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com Undersampled data...


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_41 (Dense)                │ (None, 256)            │     1,280,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,539 (5.01 MB)

 Trainable params: 1,313,539 (5.01 MB)

 Non-trainable params: 0 (0.00 B)


Treinando MLP com dados Undersampled...
Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.4307 - loss: 1.0778 - val_accuracy: 0.0029 - val_loss: 1.3136
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.6523 - loss: 0.8858 - val_accuracy: 0.8275 - val_loss: 0.6526
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8600 - loss: 0.4351 - val_accuracy: 0.8392 - val_loss: 0.4701
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9193 - loss: 0.2447 - val_accuracy: 0.7749 - val_loss: 0.5924
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.9554 - loss: 0.1502 - val_accuracy: 0.8480 - val_loss: 0.4409
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.9654 - loss: 0.1261 - val_accuracy: 0.7719 - val_loss: 0.6591
Epoch 7/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.9808 - loss: 0.0767 - val_accuracy: 0.7836 - val_loss: 0.7128
Epoch 8/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.9858 

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

# --- Pré-processamento para CNN ---

# O TfidfVectorizer não é ideal para CNNs, precisamos de uma representação baseada em sequências/embeddings.

# 1. Tokenização para CNN (Sequências de Inteiros)
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['processed_text'])

sequences = tokenizer.texts_to_sequences(df['processed_text'])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded:")
print(padded_sequences[:5])
print("\nForma das sequências padded:", padded_sequences.shape)

# --- prepara labels para CNN ---
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(df['class'])

print("\nLabels originais:", df['class'].unique())
print("Labels codificadas:", encoded_y)

# --- divisão dos dados para CNN (usando as sequências e labels codificadas) ---
X_cnn = padded_sequences
y_cnn = encoded_y

# divisão original em treino e teste para a CNN
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_cnn, test_size=0.20, random_state=42)

print("\nForma dos dados de treino para CNN (antes do undersampling):", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de teste para CNN:", X_test_cnn.shape, y_test_cnn.shape)
print("\nDistribuição das classes no treino da CNN (antes do balanceamento):")
print(pd.Series(y_train_cnn).value_counts())


# --- aplica Undersampling Aleatório nos dados de TREINO da CNN ---
try:
    # inicializa RandomUnderSampler
    # sampling_strategy='auto' remove instâncias da classe majoritária para igualar a classe minoritária
    undersampler_cnn = RandomUnderSampler(sampling_strategy='auto', random_state=42)

    X_train_cnn_resampled_under, y_train_cnn_resampled_under = undersampler_cnn.fit_resample(X_train_cnn, y_train_cnn)

    print("\nForma dos dados de treino da CNN após Undersampling:", X_train_cnn_resampled_under.shape, y_train_cnn_resampled_under.shape)
    print("\nDistribuição das classes no treino da CNN (depois do Undersampling):")
    print(pd.Series(y_train_cnn_resampled_under).value_counts())

except NameError:
    print("Erro: imblearn.under_sampling.RandomUnderSampler não encontrado. Certifique-se de que imbalanced-learn está instalado e RandomUnderSampler foi importado.")
    raise # relança o erro


# --- construção do modelo da CNN
embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

model_cnn_under = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model_cnn_under.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_cnn_under.summary()

# --- treino do Modelo CNN (Com dados Undersampled) ---
epochs = 10
batch_size = 32

print("\nTreinando CNN com dados Undersampled...")
history_cnn_under = model_cnn_under.fit(X_train_cnn_resampled_under, y_train_cnn_resampled_under,
                                        epochs=epochs,
                                        batch_size=batch_size,
                                        validation_split=0.1,
                                        verbose=1)


# --- avaliação do Modelo CNN ---
print("\nAvaliando CNN (Undersampling) no conjunto de teste original...")

loss_cnn_under, accuracy_cnn_under = model_cnn_under.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f"\nAcurácia da CNN (Undersampling) no conjunto de teste original: {accuracy_cnn_under:.4f}")

# classification report para CNN (Undersampling)
cnn_under_predictions_probs = model_cnn_under.predict(X_test_cnn)
cnn_under_predictions = tf.argmax(cnn_under_predictions_probs, axis=1).numpy()

print("\nAvaliação completa da CNN (Undersampling):")
print(classification_report(y_test_cnn, cnn_under_predictions))

# cálculo de F1-score ponderado e AUC para a CNN (Undersampling)
cnn_under_f1_weighted = f1_score(y_test_cnn, cnn_under_predictions, average='weighted')

try:
    cnn_under_auc = roc_auc_score(y_test_cnn, cnn_under_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    cnn_under_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (CNN Undersampling):", cnn_under_f1_weighted)
print("AUC (ponderado, one-vs-rest) (CNN Undersampling):", cnn_under_auc)

# dicionário de resultados
try:
    cnn_under_report = classification_report(y_test_cnn, cnn_under_predictions, output_dict=True)
    results['CNN (Embedding+Seq, Undersampling)'] = {
        'accuracy': accuracy_score(y_test_cnn, cnn_under_predictions),
        'precision (macro)': cnn_under_report['macro avg']['precision'],
        'recall (macro)': cnn_under_report['macro avg']['recall'],
        'f1-score (macro)': cnn_under_report['macro avg']['f1-score'],
        'f1-score (weighted)': cnn_under_f1_weighted,
        'auc (weighted_ovr)': cnn_under_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da CNN (Undersampling) não foram armazenados.")


Exemplo de sequências padded:
[[   3  100  838  920 2932  204   20   38   84   71   17    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   3   96   78 8148 5799   37 2045   78    4  457  381    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_7          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando CNN com dados Undersampled...
Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.4589 - loss: 1.0203 - val_accuracy: 0.9357 - val_loss: 0.5410
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.8182 - loss: 0.4934 - val_accuracy: 0.9240 - val_loss: 0.3105
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.9094 - loss: 0.2588 - val_accuracy: 0.8392 - val_loss: 0.5293
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.9585 - loss: 0.1347 - val_accuracy: 0.8450 - val_loss: 0.5123
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.9839 - loss: 0.0612 - val_accuracy: 0.8216 - val_loss: 0.5930
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9943 - loss: 0.0283 - val_accuracy: 0.8275 - val_loss: 0.6535
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.9962 - loss: 0.0191 - val_accuracy: 0.8480 - val_loss: 0.5837
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9976 

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np

# --- reutiliza dados preparados para a CNN ---

# --- aplica Undersampling Aleatório nos dados de TREINO da LSTM (que são os mesmos da CNN) ---

print("\nDistribuição das classes no treino da LSTM (antes do balanceamento):")
print(pd.Series(y_train_cnn).value_counts())

try:
    # inicializa RandomUnderSampler
    undersampler_lstm = RandomUnderSampler(sampling_strategy='auto', random_state=42)

    # aplica Undersampling apenas nos dados de TREINO da LSTM
    X_train_lstm_resampled_under, y_train_lstm_resampled_under = undersampler_lstm.fit_resample(X_train_cnn, y_train_cnn)

    print("\nForma dos dados de treino da LSTM após Undersampling:", X_train_lstm_resampled_under.shape, y_train_lstm_resampled_under.shape)
    print("\nDistribuição das classes no treino da LSTM (depois do Undersampling):")
    print(pd.Series(y_train_lstm_resampled_under).value_counts())

except NameError:
    print("Erro: imblearn.under_sampling.RandomUnderSampler não encontrado. Certifique-se de que imbalanced-learn está instalado e RandomUnderSampler foi importado.")
    raise # relança o erro


# --- Constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

model_lstm_under = Sequential([
    # camada de Embedding: Reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # camada LSTM
    LSTM(128),

    # camada de Dropout
    Dropout(0.5),

    # camada Densa (Fully Connected)
    Dense(64, activation='relu'), # Camada densa adicional
    Dropout(0.5),

    # camada de saída (igual à CNN)
    Dense(len(label_encoder.classes_), activation='softmax')
])


model_lstm_under.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model_lstm_under.summary()

# --- treino do Modelo LSTM (Com dados Undersampled) ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM com dados Undersampled...")
history_lstm_under = model_lstm_under.fit(X_train_lstm_resampled_under, y_train_lstm_resampled_under,
                                          epochs=epochs_lstm,
                                          batch_size=batch_size_lstm,
                                          validation_split=0.1,
                                          verbose=1)


# --- avaliação do Modelo LSTM ---
print("\nAvaliando LSTM (Undersampling) no conjunto de teste original...")

loss_lstm_under, accuracy_lstm_under = model_lstm_under.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print(f"\nAcurácia da LSTM (Undersampling) no conjunto de teste original: {accuracy_lstm_under:.4f}")

# classification report para LSTM (Undersampling)
lstm_under_predictions_probs = model_lstm_under.predict(X_test_cnn)
lstm_under_predictions = tf.argmax(lstm_under_predictions_probs, axis=1).numpy()
print("\nAvaliação completa da LSTM (Undersampling):")

class_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test_cnn, lstm_under_predictions, target_names=class_names))

# cálculo de F1-score ponderado e AUC para a LSTM (Undersampling)
lstm_under_f1_weighted = f1_score(y_test_cnn, lstm_under_predictions, average='weighted')

try:
    lstm_under_auc = roc_auc_score(y_test_cnn, lstm_under_predictions_probs, multi_class='ovr', average='weighted')
except Exception as e:
    lstm_under_auc = f"N/A (problema ao calcular AUC: {e})"

print("F1-score ponderado (LSTM Undersampling):", lstm_under_f1_weighted)
print("AUC (ponderado, one-vs-rest) (LSTM Undersampling):", lstm_under_auc)


print("="*50)

# dicionário de resultados
try:
    lstm_under_report = classification_report(y_test_cnn, lstm_under_predictions, output_dict=True, target_names=class_names)
    results['LSTM (Embedding+Seq, Undersampling)'] = {
        'accuracy': accuracy_score(y_test_cnn, lstm_under_predictions),
        'precision (macro)': lstm_under_report['macro avg']['precision'],
        'recall (macro)': lstm_under_report['macro avg']['recall'],
        'f1-score (macro)': lstm_under_report['macro avg']['f1-score'],
        'f1-score (weighted)': lstm_under_f1_weighted,
        'auc (weighted_ovr)': lstm_under_auc
    }
except NameError:
    print("Dicionário 'results' não encontrado. Os resultados da LSTM (Undersampling) não foram armazenados.")


Distribuição das classes no treino da LSTM (antes do balanceamento):
1    15358
2     3328
0     1140
Name: count, dtype: int64

Forma dos dados de treino da LSTM após Undersampling: (3420, 100) (3420,)

Distribuição das classes no treino da LSTM (depois do Undersampling):
0    1140
1    1140
2    1140
Name: count, dtype: int64

Construindo o Modelo LSTM...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Treinando LSTM com dados Undersampled...
Epoch 1/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 25s 209ms/step - accuracy: 0.3655 - loss: 1.0925 - val_accuracy: 0.0000e+00 - val_loss: 1.3191
Epoch 2/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 198ms/step - accuracy: 0.3719 - loss: 1.0876 - val_accuracy: 0.0000e+00 - val_loss: 1.2524
Epoch 3/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 21s 205ms/step - accuracy: 0.3779 - loss: 1.0883 - val_accuracy: 0.0000e+00 - val_loss: 1.2695
Epoch 4/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 21s 204ms/step - accuracy: 0.3729 - loss: 1.0843 - val_accuracy: 0.0000e+00 - val_loss: 1.2767
Epoch 5/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 20s 199ms/step - accuracy: 0.3586 - loss: 1.0873 - val_accuracy: 0.0000e+00 - val_loss: 1.3320
Epoch 6/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 22s 215ms/step - accuracy: 0.3750 - loss: 1.0915 - val_accuracy: 0.0000e+00 - val_loss: 1.2613
Epoch 7/15
97/97 ━━━━━━━━━━━━━━━━━━━━ 41s 217ms/step - accuracy: 0.3658 - loss: 1.0884 - val_accuracy: 0.0000e+00 - val_loss: 1.2891
Epoch 8/15
97/97 ━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

### **Resultados**

#### **Dataset Original**

| Modelo                        | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)     |
|-------------------------------|----------|---------------------|---------------------------|
| Regressão Logística           | 0.8923   | 0.8784              | 0.9387                    |
| Naive Bayes Multinomial       | 0.8322   | 0.7882              | 0.8860                    |
| SVM (Kernel Linear)           | 0.8983   | 0.8824              | 0.9361 |
| Random Forest                 | 0.8911   | 0.8769              | 0.9392                    |
| LightGBM                      | 0.8963   | 0.8866              | 0.9453                    |
| MLP (Perceptron Multicamadas) | 0.8500   | 0.8464              | 0.8944                    |
| CNN (Convolutional NN)        | 0.8700   | 0.8674              | 0.9170                    |
| LSTM                          | 0.7700   | 0.6740              | 0.5010 *(classe 0 e 2 sem predição)* |


#### **Dataset Balanceado**

##### **Quadro Resumo dos Modelos com `class_weight`**

| Modelo                  | Acurácia  | F1-score Ponderado | AUC (ponderado, OVR)    |
| ----------------------- | --------- | ------------------ | ----------------------- |
| Regressão Logística     | 0.8376     | 0.8538              | 0.9313                   |
| Naive Bayes Multinomial | 0.8322     | 0.7882              | 0.8860                   |
| SVM (Kernel Linear)     | 0.8374     | 0.8572              | 0.9409 |
| Random Forest           | 0.8907 | 0.8802          | 0.9394              |
| LightGBM                | 0.8485     | 0.8646              | 0.9386                   |
| MLP (Rede Neural)       | 0.8300     | 0.8420              | 0.8977                   |
| CNN (Rede Neural)       | 0.8300     | 0.8455              | 0.9078                   |
| LSTM (Rede Neural)      | 0.0600     | 0.0065              | 0.5994                   |


##### **Quadro Resumo dos Modelos com Oversampling**

| Modelo                  | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)    |
| ----------------------- | -------- | ------------------ | ----------------------- |
| Regressão Logística     | 0.8396    | 0.8572              | 0.9266                  |
| Naive Bayes Multinomial | 0.7906    | 0.8159              | 0.8967                  |
| SVM (Kernel Linear)     | 0.8363    | 0.8576              | 0.9238 |
| Random Forest           | 0.8757    | 0.8787              | 0.9341                  |
| LightGBM                | 0.8723    | 0.8811              | 0.9389                  |
| MLP (Rede Neural densa) | 0.8600    | 0.8456              | 0.8815                  |
| CNN                     | 0.7300    | 0.7706              | 0.8580                  |
| LSTM                    | 0.7700   | 0.6741              | 0.5000                  |



##### **Quadro Resumo dos Modelos com Undersampling**

| Modelo                  | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)    |
| ----------------------- | -------- | ------------------ | ----------------------- |
| Regressão Logística     | 0.7952   | 0.8222             | 0.9177                  |
| Naive Bayes Multinomial | 0.7331   | 0.7810             | 0.8886                  |
| SVM (Kernel Linear)     | 0.7906   | 0.8212             | 0.9259 |
| Random Forest           | 0.8158   | 0.8404             | 0.9306                  |
| LightGBM            | 0.7779   | 0.8142             | 0.9212                  |
| MLP                     | 0.6500   | 0.7087             | 0.8419                  |
| CNN                     | 0.7400   | 0.7895             | 0.8975                  |
| LSTM                    | 0.7700   | 0.6741             | 0.4985                  |
